# カード問題

## 問
10枚のカードに，それぞれ1から10までの番号が書いてある。
これらのグループを二つのグループA,Bに分けて，グループAのカード番号の総和を36,グルーブBの番号のカードの積が360となるようなグループ分けを行いたい。

In [1]:
from gaga.cga.component import *
from gaga.cga.genetic import GA

In [2]:
Affiliation=defBase(className="Affiliation",idLabel="id",typeLabel="type",letterLabel="group")     
Card=defCodon(className="Card",idLabel="id",typeLabel="value",basisLabel="affiliation")
Grouping=defIndividual(className="Grouping",idLabel="id",geneLabel="cards",scoreLabel="score")

In [3]:
from random import randrange
class CardGroupingInitializer(Creator):
    def create(self,popSize):
        return self.createPopulation(popSize)
    def giveAffilation(self):
        groups=("A","B")
        return Affiliation(None,None,"A" if randrange(0,len(groups))==0 else "B")
    def createCards(self):
        numbers=(1,2,3,4,5,6,7,8,9,10)
        return [Card(i,numbers[i],self.giveAffilation()) for i in range(len(numbers))]
    def createGroupings(self,size):    
        return [Grouping(i,self.createCards(),None) for i in range(size)]
    def createPopulation(self,popSize):
        return Population(popSize,0,self.createGroupings(popSize))

In [4]:
class CardGroupingEvaluator(Evaluator):
    def evaluate(self,grouping) -> float:
        sum_group_A=0.0
        prod_group_B=1.0
        for card in grouping.cards:
            group=card.affiliation.group
            value=float(card.value)
            if group=="A":
                sum_group_A=sum_group_A+value
            elif group=="B":
                prod_group_B=prod_group_B*value
            else:
                print("Error")
        return abs(36-sum_group_A)+abs(360-prod_group_B)

    

In [57]:
class CardProblemSolver(GA):
    def evaluate_residual(self):
        return [(grouping.id,self.evaluator.evaluate(grouping)) for grouping in self.population.individuals]

    def evaluate(self):
        res=self.evaluate_residual()
        res.sort(key=lambda x:x[1])
        popSize=self.population.size
        total=popSize*(popSize+1)/2.0
        fitness=[(res[i][0],(popSize-i)/total) for i in range(popSize)]
        fitness.sort()
        print(fitness)
        for i in range(len(fitness)):
            self.population.individuals[i].score=fitness[i][1]
        for grouping in self.population.individuals:
            print(grouping.id,[card.affiliation.group for card in grouping.cards],grouping.score)

    def step(self):
        print(self.generation)

In [58]:
cgi=CardGroupingInitializer()
cge=CardGroupingEvaluator()
cps=CardProblemSolver(cgi,None,None,None,cge,10)

In [59]:
cps.run(10)

[(0, 0.05454545454545454), (1, 0.18181818181818182), (2, 0.01818181818181818), (3, 0.03636363636363636), (4, 0.09090909090909091), (5, 0.16363636363636364), (6, 0.10909090909090909), (7, 0.14545454545454545), (8, 0.07272727272727272), (9, 0.12727272727272726)]
0 ['A', 'A', 'A', 'B', 'A', 'A', 'A', 'B', 'B', 'B'] 0.05454545454545454
1 ['B', 'A', 'A', 'A', 'B', 'A', 'A', 'B', 'A', 'B'] 0.18181818181818182
2 ['B', 'B', 'B', 'A', 'A', 'B', 'B', 'A', 'B', 'B'] 0.01818181818181818
3 ['B', 'A', 'B', 'B', 'B', 'A', 'B', 'A', 'B', 'A'] 0.03636363636363636
4 ['B', 'A', 'A', 'B', 'A', 'B', 'B', 'A', 'B', 'A'] 0.09090909090909091
5 ['A', 'B', 'B', 'A', 'A', 'A', 'A', 'B', 'B', 'A'] 0.16363636363636364
6 ['A', 'A', 'A', 'B', 'B', 'A', 'A', 'B', 'B', 'A'] 0.10909090909090909
7 ['B', 'B', 'B', 'A', 'A', 'A', 'A', 'B', 'A', 'A'] 0.14545454545454545
8 ['A', 'A', 'B', 'A', 'A', 'B', 'A', 'A', 'B', 'B'] 0.07272727272727272
9 ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B'] 0.12727272727272726
0
[(0, 0.